### Load according data

In [14]:
import pandas as pd
import oemof.solph as solph


#restore and load energy system
energy_system = solph.EnergySystem()
energy_system.restore('C:\\Users\\ann82611\\ownCloud\\U-Platte\\04_Code\\hydrogen_hub\\h2_hub_minimalschnitt\\h2_hub_dumps', 'h2_hub_dump.oemof')

results_main = energy_system.results['main']
results_meta = energy_system.results['meta'] #for costs


# Convert the results to a DataFrame
results = pd.Series(results_main)
results_meta_pd = pd.Series(results_meta)
print(results.info())
#print(results.describe())



<class 'pandas.core.series.Series'>
MultiIndex: 14 entries, ("<oemof.solph.buses._bus.Bus: 'co2_emissions_bus'>", "<oemof.solph.components._sink.Sink: 'co2_emissions'>") to ("<oemof.solph.components._source.Source: 'wind'>", "<oemof.solph.buses._bus.Bus: 'electricity'>")
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
14 non-null     object
dtypes: object(1)
memory usage: 908.0+ bytes
None


## System Performance
The system performance is defined per below consisting of:
* Electricity deliverd to steel mill
* Hydrogen deliverd to steel mill
* Co2 Emissions
* Costs #aufsummierte kosten

Please keep in mind that the definition of the system performance may vary over time and also to the considered system accordingly.

In [12]:
# for key in results_main.keys():
#     print(key)
#Schlüssel sind objekte (keine Strings!) 

# get flows
flow_el_steel = results.loc[(energy_system.node['electricity'], energy_system.node['steel_mill'])]['sequences']
print(flow_el_steel)


costs = results_meta_pd['objective']
print(costs)

variable_name              flow
2019-01-01 00:00:00  57000000.0
2019-01-01 01:00:00  57000000.0
2019-01-01 02:00:00  57000000.0
2019-01-01 03:00:00  57000000.0
2019-01-01 04:00:00  57000000.0
...                         ...
2019-12-31 20:00:00  57000000.0
2019-12-31 21:00:00  57000000.0
2019-12-31 22:00:00  57000000.0
2019-12-31 23:00:00  57000000.0
2020-01-01 00:00:00         NaN

[8761 rows x 1 columns]
85715871284500.0


c:\Users\ann82611\AppData\Local\Programs\Python\Python312\Lib\site-packages\oemof\network\energy_system.py:205: ExperimentalFeatureWarning: The API to access nodes by label is experimental and might change without prior notice.
  warnings.warn(msg, debugging.ExperimentalFeatureWarning)


In [49]:
from oemof.solph import views
from oemof.solph.components import Sink

# views.convert_keys_to_strings(results)




# for key in results.keys():
#     print(key)
# print(results[('electricity','steel_mill')]['sequences'])

# flows = [x for x in results.keys() if x[1] is not None]
# nodes = [x for x in results.keys() if x[1] is None]

# for node in nodes:
#     print(f"Node: {node}")

# for flow in flows:
#     print(flow)
# # # Access the flow between electricity and steel_mill directly
# flow_el_steel = results_main[
#     ("<oemof.solph.buses._bus.Bus: 'electricity'>", "<oemof.solph.components._sink.Sink: 'steel_mill'>")
# ]['sequences']

# print(flow_el_steel)



("<oemof.solph.buses._bus.Bus: 'electricity'>", "<oemof.solph.components._sink.Sink: 'electricity_slack'>")
("<oemof.solph.buses._bus.Bus: 'electricity'>", "<oemof.solph.components._converter.Converter: 'electrolyzer'>")
("<oemof.solph.buses._bus.Bus: 'electricity'>", "<oemof.solph.components._sink.Sink: 'steel_mill'>")
("<oemof.solph.components._converter.Converter: 'electrolyzer'>", "<oemof.solph.buses._bus.Bus: 'h2_to_storage'>")
("<oemof.solph.components._source.Source: 'grid'>", "<oemof.solph.buses._bus.Bus: 'electricity'>")
("<oemof.solph.components._source.Source: 'h2_ship'>", "<oemof.solph.buses._bus.Bus: 'h2_to_storage'>")
("<oemof.solph.components._generic_storage.GenericStorage: 'h2_storage'>", nan)
("<oemof.solph.components._generic_storage.GenericStorage: 'h2_storage'>", "<oemof.solph.buses._bus.Bus: 'h2_to_production'>")
("<oemof.solph.buses._bus.Bus: 'h2_to_production'>", "<oemof.solph.components._sink.Sink: 'steel_mill'>")
("<oemof.solph.buses._bus.Bus: 'h2_to_storage'>

In [51]:
electricity_bus = views.node(results_main, 'electricity')
print(electricity_bus['sequences'][('electricity', 'steel_mill')])

KeyError: ('electricity', 'steel_mill')